In [1]:
from MQGO import data, main, ML_engine, optimizer, QC_engine 
from deepmd.calculator import DP
import subprocess
import numpy as np
import json

2022-04-23 18:44:46.325293: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-23 18:44:46.325432: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
d_steps = [3000]
tot_steps = [4000]
subprocess.run('cp water0.xyz water.xyz', shell=True)

index = 0
for ii in range(0,len(d_steps)):
    for jj in range(0,len(tot_steps)):
        js_setting = {}
        with open('./input0.json', 'r') as file:
            json_setting = json.load(file)
            print('after remake json: ',json_setting['learning_rate']['decay_steps'])
            
        json_setting['learning_rate']['decay_steps'] = d_steps[ii]
        json_setting['training']['numb_steps'] = tot_steps[jj]
        print('ii,jj:',ii,jj)
        print('before make json: ',json_setting['learning_rate']['decay_steps'])
        with open('./input.json', 'w') as js:
            json.dump(json_setting, js, indent=4)
        
        setting = {'basis':'6-31g', 'spin':0, 'xc':None, 'restricted':True, 'verbose':0}
        geo = main.MLgeomopt(qc_engine='pyscf', ml_engine='deepmd', work_path='./', xyz_path='./water.xyz',**setting)
        geo.opt_algorithm = 'lbfgs'
        geo.debug = True
        geo.kernel()
        
        Opt = optimizer.Optimizer(xyz_path="water0.xyz",
									work_path="./",
									ml_engine='deepmd',
									outter_cycle=1,
									algorithm='lbfgs',
									max_opt_cycle=100)
        atom0 = Opt.ase_read_xyz()
        atom0.calc = DP(model='graph.pb')
        print(atom0.get_potential_energy())
        force0_ml = atom0.get_forces()

        forces = np.loadtxt('raw/force.raw')
        force0_qc = forces[0].reshape((6,3))

        delta = force0_qc-force0_ml
        with open('debug_diff_force.txt', 'a') as file:
            file.write('24.18f\n'%atom0.get_potential_energy())
            for i in range(0, len(force0_qc)):
                for j in range(0, 3):
                    file.write('%24.18f'%delta[i][j])
                file.write('\n')
            file.write('\n')
            
        print('after dp: ',json_setting['learning_rate']['decay_steps'])
        subprocess.run('cp water0.xyz water.xyz', shell=True)
        # subprocess.run('rm input.json', shell=True)

after remake json:  4000
ii,jj: 0 0
before make json:  3000


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-12
SCF conv_tol_grad = None
SCF max_cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpnoe8grd1
max_memory 21438 MB (current use 360 MB)
Set gradient conv threshold to 1e-06
init E= -151.415122555484
  HOMO = -0.458914611898626  LUMO = 0.101611688525233
cycle= 1 E= -151.868630726732  delta_E= -0.454  |g|= 0.597  |ddm|= 1.64
  HOMO = -0.400323409471753  LUMO = 0.194953131082893
cycle= 2 E= -151.927771207762  delta_E= -0.0591  |g|= 0.357  |ddm|= 0.608
  HOMO = -0.499862289875179  LUMO = 0.180933357043683
cycle= 3 E= -151.95036021667  delta_E= -0.0226  |g|= 0.0355  |ddm|= 0.229


OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-11
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #216: KMP_AFFINITY: x2APIC ids not unique.
OMP: Info #216: KMP_AFFINITY: decoding legacy APIC ids.
OMP: Info #216: KMP_AFFINITY: legacy APIC ids not unique.
OMP: Info #220: KMP_AFFINITY: parsing /proc/cpuinfo.
OMP: Info #157: KMP_AFFINITY: 12 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #287: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #192: KMP_AFFINITY: 1 socket x 6 cores/socket x 2 threads/core (6 total cores)
OMP: Info #218: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 1 maps to socket 0 core 0 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 2 maps to socket 0 core 1 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 3 maps to socket 0 core 1 thread 1 
OMP: Info #172: KMP_AFFINITY: OS p

  HOMO = -0.497612927938677  LUMO = 0.187865526656965
cycle= 4 E= -151.950856808118  delta_E= -0.000497  |g|= 0.00594  |ddm|= 0.0375
  HOMO = -0.498863887981174  LUMO = 0.187642464715898
cycle= 5 E= -151.950875581763  delta_E= -1.88e-05  |g|= 0.000688  |ddm|= 0.00925
  HOMO = -0.498920556933575  LUMO = 0.187620019572829
cycle= 6 E= -151.950875817958  delta_E= -2.36e-07  |g|= 0.000198  |ddm|= 0.00111
  HOMO = -0.498975213195492  LUMO = 0.187616477545118
cycle= 7 E= -151.950875827637  delta_E= -9.68e-09  |g|= 5.74e-05  |ddm|= 0.000157
  HOMO = -0.498963904179647  LUMO = 0.187615348330252
cycle= 8 E= -151.950875829917  delta_E= -2.28e-09  |g|= 1.17e-05  |ddm|= 0.00012
  HOMO = -0.498961965070038  LUMO = 0.187614886419037
cycle= 9 E= -151.95087583003  delta_E= -1.13e-10  |g|= 1.35e-06  |ddm|= 3.23e-05
  HOMO = -0.498961881531363  LUMO = 0.187614670626288
cycle= 10 E= -151.950875830031  delta_E= -6.25e-13  |g|= 2.51e-07  |ddm|= 2.62e-06
  HOMO = -0.498961918333598  LUMO = 0.18761465484597
E

2022-04-23 18:44:49.726100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-23 18:44:49.726245: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
2022-04-23 18:44:54.339216: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 18:44:54.353064: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed cal

optimizer.py 77: [[ 3.63802  1.5711   1.76   ]
 [26.36198  1.5711   1.76   ]
 [ 3.95828  2.55412  1.74392]
 [ 2.63613  1.66728  1.76493]
 [26.09174  0.99418  2.56104]
 [26.0955   0.9908   0.96072]]


...

optimizer.py 77: [[ 3.4108122   1.93083059  1.75628335]
 [26.18307436  1.1853622   1.76058615]
 [ 3.64281315  2.82664577  1.74200653]
 [ 3.17880466  1.03502364  1.77056009]
 [26.21213255  1.25463622  2.68300952]
 [26.1540131   1.11608157  0.83816433]]
       Step     Time          Energy          fmax
LBFGS:    0 19:15:13    -4132.994068   4.52172e-02
LBFGS:    1 19:15:13    -4132.994238   8.79810e-02
LBFGS:    2 19:15:13    -4132.994006   2.03136e-03
LBFGS:    3 19:15:13    -4132.994007   7.02991e-03
LBFGS:    4 19:15:13    -4132.994006   2.03404e-05


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-12
SCF conv_tol_grad = None
SCF max_cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpsszcyk9k
max_memory 21438 MB (current use 1486 MB)
Set gradient conv threshold to 1e-06
init E= 17.

2022-04-23 19:15:15.074504: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-23 19:15:15.074674: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
2022-04-23 19:15:19.690768: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 19:15:19.704910: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed cal

optimizer.py 77: [[ 3.41081254  1.9308327   1.75628332]
 [26.1830743   1.18536207  1.76058648]
 [ 3.64264213  2.82598505  1.74201706]
 [ 3.17897534  1.03568225  1.77054959]
 [26.21211118  1.25458527  2.68233008]
 [26.15403453  1.11613265  0.83884343]]
       Step     Time          Energy          fmax
LBFGS:    0 19:18:45    -4132.827365   3.12275e-02
LBFGS:    1 19:18:45    -4132.827448   6.10399e-02
LBFGS:    2 19:18:45    -4132.827336   1.15057e-04


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-12
SCF conv_tol_grad = None
SCF max_cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpmezvgmjh
max_memory 21438 MB (current use 1619 MB)
Set gradient conv threshold to 1e-06
init E= 17.6258433530698
  HOMO = -8.36035915440123  LUMO = -4.79104427134262
cycle= 1 E= -139.485306387996  delt

2022-04-23 19:18:46.508125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-23 19:18:46.508300: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
2022-04-23 19:18:51.652026: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 19:18:51.666503: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed cal

optimizer.py 77: [[ 3.41081248  1.93083256  1.75628332]
 [26.18307429  1.18536205  1.76058646]
 [ 3.64275931  2.82643744  1.74200985]
 [ 3.17885822  1.03523     1.7705568 ]
 [26.21212586  1.25462026  2.68279586]
 [26.15401986  1.11609768  0.83837767]]
       Step     Time          Energy          fmax
LBFGS:    0 19:22:11    -4133.025628   3.69953e-02
LBFGS:    1 19:22:11    -4133.025743   7.20472e-02
LBFGS:    2 19:22:11    -4133.025587   2.13874e-04


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-12
SCF conv_tol_grad = None
SCF max_cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpd98q3qdz
max_memory 21438 MB (current use 1682 MB)
Set gradient conv threshold to 1e-06
init E= 17.6371516192903
  HOMO = -8.3609527933737  LUMO = -4.79194615965879
cycle= 1 E= -139.487326708187  delta